In [1]:
%load_ext autoreload
%autoreload 2
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertConfig
import torch

In [27]:
config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states = True, output_attentions = True, 
                                    return_dict = True) # Note output_attentions
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', config = config)
model = BertForMaskedLM.from_pretrained('bert-base-uncased', config = config)
model = model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
config.output_attentions

True

In [29]:
sentence = "attention is awesome."

In [30]:
indexed_tokens = tokenizer.encode(sentence, add_special_tokens = True)
#indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
bert_tokens = tokenizer.convert_ids_to_tokens(indexed_tokens)
tokens_tensor = torch.tensor([indexed_tokens])

In [48]:
with torch.no_grad():
    result = model(tokens_tensor)
print(result.keys())

odict_keys(['logits', 'hidden_states', 'attentions'])


In [49]:
att = result["attentions"]
print(len(att)) # 12 - 1 for each layer

12


In [50]:
# let's look on the attention at layer 5.
# each attention layer is a tensor of shape (batch_size, num_heads, sequence_length, sequence_length)

att5 = att[5]
print(att5.shape)

torch.Size([1, 12, 6, 6])


In [51]:
# let's inspect see how the first word ("attention") attends to all other words by the 3rd attention head.
head_ind = 3
sent_ind = 0 # a single sentence in the batch
word_index = 1 # "attention" appears after the CLS token

att_distribution_for_the_word_attention_in_layer_5_head_3 = att5[sent_ind][head_ind][word_index]
print(att_distribution_for_the_word_attention_in_layer_5_head_3)
print(torch.sum(att_distribution_for_the_word_attention_in_layer_5_head_3))

tensor([0.0118, 0.0247, 0.0127, 0.0085, 0.0053, 0.9370])
tensor(1.)
